In [1]:
import dynamic_traffic_signal
import importlib

In [6]:
importlib.reload(dynamic_traffic_signal)

<module 'dynamic_traffic_signal' from '/home/utkarsh/Study/Quantela/Traffic_Simulation/dynamic_traffic_signal.py'>

In [2]:
a = dynamic_traffic_signal.DynamicTrafficSignal('Testing_class_ijipura')

In [3]:
a.optimize_traffic_lights(timing_range={'min':18, 'max':21}, gui=True)

18 18
 Retrying in 1 seconds
18 19
 Retrying in 1 seconds
18 20
 Retrying in 1 seconds
19 18
 Retrying in 1 seconds
19 19
 Retrying in 1 seconds
19 20
 Retrying in 1 seconds
20 18
 Retrying in 1 seconds
20 19
 Retrying in 1 seconds
20 20
 Retrying in 1 seconds
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:48415 [Errno 111] Connection refused
 Retrying in 2 seconds


{'one': 18, 'two': 20}

In [4]:
a.optimized_result

{'one': 18, 'two': 20}